In [ ]:
# Установка необходимых версий для blocksnet
!pip install numpy==1.23.5 pyarrow==12.0.0 scikit-learn==1.2.2 matplotlib==3.7.1

# Установка остальных библиотек
!pip install osmnx contextily mapclassify iduedu blocksnet

# Импорт библиотек
import osmnx as ox         # Библиотека для работы с OpenStreetMap данными
import geopandas as gpd     # Библиотека для работы с геопространственными данными
import pandas as pd         # Библиотека для работы с табличными данными
import numpy as np          # Библиотека для числовых вычислений
import matplotlib.pyplot as plt  # Библиотека для визуализации данных
from shapely.ops import unary_union  # Функция для объединения геометрий
import contextily as ctx    # Библиотека для добавления фона (тайлов) к картам
import mapclassify         # Библиотека для классификации данных (например, для хлороплетных карт)
from iduedu import get_boundary, get_drive_graph  # Функции для получения границ и графа дорожной сети
from blocksnet import BlocksGenerator, BlocksSplitter, AccessibilityProcessor, Accessibility, Connectivity, Provision, ProvisionMethod, City
from blocksnet import LandUsePrediction, Diversity, Centrality, PopulationCentrality, Spacematrix, VacantArea
from shapely.geometry import Point  # Для создания точечных геометрий
from IPython.display import display  # Для отображения объектов в Jupyter Notebook

In [ ]:
# -------------------------------------------
# ЗАГРУЗКА данных
# -------------------------------------------

# Задаём список центральных районов Санкт-Петербурга, для которых будем загружать данные
central_districts = [
    "Центральный район, Санкт-Петербург, Россия",
    "Адмиралтейский район, Санкт-Петербург, Россия",
    "Василеостровский район, Санкт-Петербург, Россия",
    "Петроградский район, Санкт-Петербург, Россия"
]

# Получаем геопространственные данные (GeoDataFrame) для каждого района по его названию
districts_gdf = ox.geocode_to_gdf(central_districts)

# Объединяем полигоны всех районов в одну общую геометрию для центральной области
central_area = districts_gdf.unary_union

# Загружаем все объекты, помеченные как здания, внутри центральной области
buildings = ox.geometries_from_polygon(central_area, tags={"building": True})

# Получаем граф дорожной сети для автомобилей в пределах центральной области
street_graph = ox.graph_from_polygon(central_area, network_type='drive')

# Преобразуем граф уличной сети в GeoDataFrames для узлов (nodes) и рёбер (edges)
nodes, edges = ox.graph_to_gdfs(street_graph)

# Загружаем точки интереса (POI) внутри центральной области: рестораны, кафе, бары и гостиницы
pois = ox.geometries_from_polygon(central_area, tags={
    "amenity": ["restaurant", "cafe", "bar"],
    "tourism": ["hotel"]
})

# Загружаем объекты, связанные с историей и искусством: памятники, мемориалы и произведения искусства
monuments = ox.geometries_from_polygon(central_area, tags={
    "historic": ["monument", "memorial"],
    "tourism":  "artwork"
})

# Приводим все загруженные данные к одной проекции (EPSG:3857), которая подходит для визуализации на веб-картах
buildings = buildings.to_crs(epsg=3857)
edges = edges.to_crs(epsg=3857)
pois = pois.to_crs(epsg=3857)
monuments = monuments.to_crs(epsg=3857)
districts_gdf = districts_gdf.to_crs(epsg=3857)

In [5]:
# -------------------------------------------
# ФИЛЬТРАЦИЯ (знаковые здания, мосты, набережные, памятники)
# -------------------------------------------

# Фильтруем здания, оставляя только знаковые объекты. Критерии:
# - Наличие информации о историческом значении (historic) или наследии (heritage)
# - Значение поля tourism: "attraction" или "museum"
# - Значение поля amenity: типы, связанные с культурой и поклонением (театр, центр искусств, музей, место поклонения)
landmark_buildings = buildings[
    (
        buildings.get('historic').notna() |  # Фильтруем объекты с непустым значением historic
        buildings.get('heritage').notna() |  # Фильтруем объекты с непустым значением heritage
        buildings.get('tourism').isin(['attraction', 'museum']) |  # Фильтруем по типу tourism
        buildings.get('amenity').isin(['theatre','arts_centre','museum','place_of_worship'])  # Фильтруем по типу amenity
    )
]

# Оставляем только здания, у которых указано название (name), чтобы исключить мелкие или нерелевантные объекты
landmark_buildings = landmark_buildings[landmark_buildings.get('name').notna()].copy()

# Добавляем новый столбец 'category' для дальнейшей идентификации типа объекта (в данном случае – здание)
landmark_buildings['category'] = 'building'

# Фильтруем рёбра уличной сети для выбора мостов:
# - Условие: значение поля 'bridge' должно быть 'yes' и у моста должно быть указано название
bridges = edges[(edges.get('bridge') == 'yes') & edges.get('name').notna()].copy()
bridges['category'] = 'bridge'

# Фильтруем рёбра для выбора набережных:
# - Условие: в названии ('name') объекта должна встречаться подстрока 'набереж' (поиск без учёта регистра)
embankments = edges[edges.get('name').str.contains('набереж', case=False, na=False)].copy()
embankments['category'] = 'embankment'

# Фильтруем объекты памятников, оставляя только те, у которых указано название
monuments = monuments[monuments.get('name').notna()].copy()
monuments['category'] = 'monument'

# Объединяем все отфильтрованные категории объектов (знаковые здания, мосты, набережные и памятники)
# для дальнейшего анализа или визуализации
candidates = pd.concat([landmark_buildings, bridges, embankments, monuments],
                       ignore_index=True)

In [ ]:
# -------------------------------------------
# Расчёт факторов (Icultural, Iprox, Itour, Ienergo, Iaesthetic, Icost)
# -------------------------------------------
# Инициализируем новые столбцы для расчёта факторов с начальным значением 0.0
candidates['Icultural']  = 0.0   # Культурный фактор
candidates['Iprox']      = 0.0   # Фактор близости к важным улицам
candidates['Itour']      = 0.0   # Фактор туристической привлекательности
candidates['Ienergo']    = 0.0   # Фактор энергоэффективности (обратная функция от размера)
candidates['Iaesthetic'] = 0.0   # Эстетический фактор (экспертная оценка)
candidates['Icost']      = 0.0   # Фактор стоимости (чем выше стоимость, тем ниже балл)

# Расчёт культурного фактора (Icultural)
# Если у объекта есть признаки историчности, наследия, упоминание в wikipedia или он относится к культурным типам amenity,
# увеличиваем значение Icultural на 1 для каждого критерия
candidates.loc[candidates['historic'].notna(),  'Icultural'] += 1
candidates.loc[candidates['heritage'].notna(),  'Icultural'] += 1
candidates.loc[candidates['wikipedia'].notna(), 'Icultural'] += 1
candidates.loc[candidates['amenity'].isin(['theatre','museum','arts_centre','place_of_worship']),
               'Icultural'] += 1

# Нормализуем значение Icultural по максимальному значению, чтобы перевести его в диапазон [0,1]
max_cult = candidates['Icultural'].max()
if max_cult > 0:
    candidates['Icultural'] /= max_cult

# Фактор близости к важным улицам (Iprox)
# Выбираем основные дороги (тип highway: primary, secondary, trunk)
major_roads = edges[edges['highway'].isin(['primary','secondary','trunk'])].copy()
if len(major_roads) > 0:
    # Объединяем геометрии основных дорог в одну объединённую геометрию
    roads_union = unary_union(major_roads.geometry)
    # Вычисляем расстояния от каждого кандидата до ближайшей основной дороги
    distances = candidates.geometry.distance(roads_union)
    Dmax = 200  # Максимальное расстояние, при котором влияние учитывается
    # Фактор рассчитывается как 1 - (расстояние / Dmax), обрезая отрицательные значения до 0
    candidates['Iprox'] = 1 - (distances / Dmax)
    candidates.loc[candidates['Iprox'] < 0, 'Iprox'] = 0

# Фактор туристической привлекательности (Itour)
# Определяем радиус буфера R (300 метров) для подсчёта POI рядом с объектом
R = 300
def count_pois_in_buffer(geom):
    # Для каждой геометрии считаем количество точек интереса (POI) внутри буфера R
    return pois[pois.geometry.distance(geom) <= R].shape[0]

# Применяем функцию для каждого кандидата и записываем количество найденных POI в столбец 'poi_count'
candidates['poi_count'] = candidates.geometry.apply(count_pois_in_buffer)
# Нормализуем значение Itour, деля количество POI на максимум среди всех кандидатов
max_poi = candidates['poi_count'].max()
if max_poi > 0:
    candidates['Itour'] = candidates['poi_count'] / max_poi

# Фактор энергоэффективности (Ienergo)
# Определяем размер объекта:
# - Для полигональных объектов (Polygon, MultiPolygon) – используем площадь
# - Для линейных объектов (LineString, MultiLineString) – используем длину
# - Для точечных объектов – оставляем 0
candidates['size_val'] = 0.0
is_poly = candidates.geometry.geom_type.isin(['Polygon','MultiPolygon'])
is_line = candidates.geometry.geom_type.isin(['LineString','MultiLineString'])
is_pt   = candidates.geometry.geom_type.isin(['Point'])

candidates.loc[is_poly, 'size_val'] = candidates.loc[is_poly, 'geometry'].area
candidates.loc[is_line, 'size_val'] = candidates.loc[is_line, 'geometry'].length

# Вычисляем максимальный размер среди кандидатов
max_size = candidates['size_val'].max()
# Вычисляем Ienergo как обратную нормированную функцию от размера
if max_size > 0:
    candidates['Ienergo'] = 1 - (candidates['size_val'] / max_size)
else:
    candidates['Ienergo'] = 1

# Эстетический фактор (Iaesthetic) – экспертная оценка по категориям объектов (исходя из литературы, статей)
# Присваиваем фиксированные значения в зависимости от категории:
# - Здание: 0.8, Мост: 0.7, Набережная: 0.5, Памятник: 1.0
candidates.loc[candidates['category']=='building',   'Iaesthetic'] = 0.8
candidates.loc[candidates['category']=='bridge',     'Iaesthetic'] = 0.7
candidates.loc[candidates['category']=='embankment', 'Iaesthetic'] = 0.5
candidates.loc[candidates['category']=='monument',   'Iaesthetic'] = 1.0

In [ ]:
# -------------------------------------------
# Расчёт стоимости и преобразование в индекс Icost
# -------------------------------------------
# Инициализируем столбец cost с нулевыми значениями
candidates['cost'] = 0.0

# Рассчитываем стоимость для каждого типа объекта по заданным формулам:
# Для здания: базовая стоимость + стоимость за площадь
candidates.loc[candidates['category']=='building', 'cost'] = (
    7_000_000 + 1_000 * candidates['size_val']
)

# Для моста: базовая стоимость + стоимость за длину
candidates.loc[candidates['category']=='bridge', 'cost'] = (
    10_000_000 + 15_000 * candidates['size_val']
)

# Для набережной: базовая стоимость + стоимость за длину
candidates.loc[candidates['category']=='embankment', 'cost'] = (
    7_000_000 + 7_000 * candidates['size_val']
)

# Для памятника (точка): фиксированная стоимость
candidates.loc[candidates['category']=='monument', 'cost'] = (
    4_000_000
)

# Преобразуем рассчитанную стоимость в индекс Icost в диапазоне [0,1]
# Чем выше стоимость, тем ниже балл Icost
min_c = candidates['cost'].min()
max_c = candidates['cost'].max()
if max_c > min_c:
    candidates['Icost'] = 1 - (candidates['cost'] - min_c)/(max_c - min_c)
else:
    candidates['Icost'] = 1


In [8]:
# -------------------------------------------
# 4. Расчёт итогового Score и бюджет
# -------------------------------------------
# Определяем веса (альфа-коэффициенты) для каждого фактора.
# Сценарий 1:
alpha1 = 0.2  # Вес для Ienergo
alpha2 = 0.2  # Вес для Icultural
alpha3 = 0.2  # Вес для Icost
alpha4 = 0.2  # Вес для Iaesthetic
alpha5 = 0.2  # Вес для Itour

In [ ]:
# Сценарий 2:
alpha1 = 0.0  # Ienergo
alpha2 = 0.2  # Icultural
alpha3 = 0.0  # Icost
alpha4 = 0.6  # Iaesthetic
alpha5 = 0.2  # Itour

In [ ]:
# Сценарий 3:
alpha1 = 0.5  # Ienergo
alpha2 = 0.05  # Icultural
alpha3 = 0.4  # Icost
alpha4 = 0.05  # Iaesthetic
alpha5 = 0.0  # Itour

In [ ]:
# Итоговый Score рассчитывается как взвешенная сумма всех факторов
candidates['Score'] = (
    alpha1 * candidates['Ienergo'] +
    alpha2 * candidates['Icultural'] +
    alpha3 * candidates['Icost']    +
    alpha4 * candidates['Iaesthetic'] +
    alpha5 * candidates['Itour']
)

# Сортируем кандидатов по итоговому Score в порядке убывания
candidates = candidates.sort_values('Score', ascending=False).reset_index(drop=True)
print("\n=== Top 5 объектов по Score ===")
print(candidates[['name','category','Score','cost']].head(5))

In [ ]:
# -------------------------------------------
# 5. Решение задачи «рюкзака» (Knapsack) для выбора объектов при ограниченном бюджете
# -------------------------------------------
# Переводим стоимость в миллионы руб. и округляем до целых значений
candidates['cost_mln'] = (candidates['cost'] / 1e6).round().astype(int)

# Допустим, общий бюджет B = 12000 млн руб (12 млрд руб), что соответствует 3 годам
B = 1000

# Формируем списки стоимостей и значений (Score) для каждого объекта
costs  = candidates['cost_mln'].tolist()
values = candidates['Score'].tolist()
N = len(candidates)

# Реализуем динамическое программирование для задачи рюкзака: создаём таблицу размером (N+1) x (B+1)
dp = [[0.0]*(B+1) for _ in range(N+1)]

# Заполняем таблицу dp, где dp[i][b] – максимальный суммарный Score, который можно достичь,
# используя первые i объектов и бюджет b
for i in range(1, N+1):
    ci = costs[i-1]  # стоимость i-го объекта
    vi = values[i-1] # Score i-го объекта
    for b in range(B+1):
        if ci <= b:
            dp[i][b] = max(dp[i-1][b], dp[i-1][b-ci] + vi)
        else:
            dp[i][b] = dp[i-1][b]

max_value = dp[N][B]
print(f"\nМакс. суммарный Score при бюджете = {B} млн руб:", round(max_value,2))

# Восстанавливаем выбранные объекты, проходя по таблице dp в обратном порядке
chosen_idxs = []
b = B
for i in range(N, 0, -1):
    if dp[i][b] != dp[i-1][b]:
        chosen_idxs.append(i-1)  # сохраняем индекс выбранного объекта
        b -= costs[i-1]          # уменьшаем оставшийся бюджет

# Переворачиваем список индексов, чтобы получить исходный порядок
chosen_idxs.reverse()
selected_objects = candidates.iloc[chosen_idxs].copy()
selected_objects.reset_index(drop=True, inplace=True)

print("\n=== Выбранные объекты при данном бюджете ===")
print(selected_objects[['name','category','Score','cost','cost_mln']])

In [ ]:
# -------------------------------------------
# 6. Визуализация результатов
# -------------------------------------------
# Создаём первую карту с использованием CartoDB Positron в качестве базовой карты
fig, ax = plt.subplots(figsize=(12, 10))
# Отображаем районы с белым фоном и чёрными границами
districts_gdf.plot(ax=ax, facecolor='white', edgecolor='black', alpha=0.3)
# Отображаем все кандидаты серыми точками (markersize=7)
candidates.plot(ax=ax, color='grey', markersize=7, label='Все объекты-кандидаты')
# Определяем список категорий для выборки
cats = ['building','bridge','embankment','monument']
for cat in cats:
    sub = selected_objects[selected_objects['category'] == cat]
    if sub.empty:
        continue
    # Если объекты – линии (мосты, набережные), отображаем их с увеличенной толщиной линий
    if cat in ['bridge', 'embankment']:
        sub.plot(ax=ax, linewidth=5, label=f"{cat} (selected)")
    else:
        # Для полигональных объектов (здания, памятники) отображаем их центроиды в виде красных точек
        centroids = sub.geometry.centroid
        ax.scatter(centroids.x, centroids.y, s=10, color='red', label=f"{cat} (selected)")
# Добавляем базовую карту
ctx.add_basemap(ax, crs=districts_gdf.crs.to_string(), source=ctx.providers.CartoDB.Positron)
ax.set_title("Демонстрация выбора объектов АХО (Knapsack) при ограниченном бюджете за 1 год (Сценарий 3)")
ax.set_xlabel("X (метры в проекции WebMercator)")
ax.set_ylabel("Y (метры в проекции WebMercator)")
plt.legend()
plt.show()

# Создаём вторую карту с использованием OpenStreetMap Mapnik в качестве базовой карты
fig, ax = plt.subplots(figsize=(12, 10))
districts_gdf.plot(ax=ax, facecolor='white', edgecolor='black', alpha=0.3)
candidates.plot(ax=ax, color='grey', markersize=7, label='Все объекты-кандидаты')
for cat in cats:
    sub = selected_objects[selected_objects['category'] == cat]
    if sub.empty:
        continue
    if cat in ['bridge', 'embankment']:
        sub.plot(ax=ax, linewidth=5, label=f"{cat} (selected)")
    else:
        centroids = sub.geometry.centroid
        ax.scatter(centroids.x, centroids.y, s=10, color='red', label=f"{cat} (selected)")
ctx.add_basemap(ax, crs=districts_gdf.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
ax.set_title("Демонстрация выбора объектов АХО (Knapsack) при ограниченном бюджете за 3 года (Сценарий 1)")
ax.set_xlabel("X (метры в проекции WebMercator)")
ax.set_ylabel("Y (метры в проекции WebMercator)")
plt.legend()
plt.show()

In [ ]:
# Создаём фигуру и ось
fig, ax = plt.subplots(figsize=(12, 10))

# Рисуем слой районов (например, districts_gdf) в качестве фона
districts_gdf.plot(ax=ax, facecolor='white', edgecolor='black', alpha=0.3)

# Вычисляем квантильную разбивку для столбца 'Score' на 5 классов
scheme = mapclassify.Quantiles(candidates['Score'], k=5)

# Рисуем слой кандидатов, раскрашенных по 'Score' с использованием классификации и colormap 'viridis'
selected_objects.plot(
    ax=ax,
    column='Score',
    scheme='quantiles',
    cmap='viridis',         # colormap, аналогичный интерактивному варианту
    legend=True,
    classification_kwds={'k': 5}
)

# Добавляем базовую карту с использованием CartoDB.Positron
ctx.add_basemap(ax, crs=candidates.crs.to_string(), source=ctx.providers.CartoDB.Positron)

# Оформляем карту
ax.set_title("Распределение Score у выбранных кандидатов (По квантилям, Сценарий 1)", fontsize=16)
ax.set_axis_off()  # убираем оси для эстетики

plt.show()





In [ ]:
fig2, ax2 = plt.subplots(figsize=(12, 10))
scheme = mapclassify.Quantiles(candidates['Score'], k=5)  # квантильная разбивка на 5 классов

districts_gdf.plot(ax=ax2, facecolor='white', edgecolor='black', alpha=0.3)
candidates.plot(
    ax=ax2,
    column='Score',
    scheme='quantiles',
    cmap='YlGnBu',  # заменили на другой colormap, например, 'YlGnBu'
    legend=True,
    classification_kwds={'k':5}
)

# Заменяем источник базовой карты на CartoDB.Positron
ctx.add_basemap(ax2, crs=candidates.crs.to_string(), source=ctx.providers.CartoDB.Positron)

ax2.set_title("Распределение Score у всех кандидатов (По квантилям)")
plt.show()

In [ ]:
# ------------------------------------------------------------------------------
# Определение центральных районов Санкт-Петербурга и получение их границ
# ------------------------------------------------------------------------------
central_districts = [
    "Центральный район, Санкт-Петербург, Россия",
    "Адмиралтейский район, Санкт-Петербург, Россия",
    "Василеостровский район, Санкт-Петербург, Россия",
    "Петроградский район, Санкт-Петербург, Россия"
]

# Получаем GeoDataFrame с границами районов по названию
districts_gdf = ox.geocode_to_gdf(central_districts)
# Для обратной совместимости используем переменную districts_gdf2
districts_gdf2 = districts_gdf.copy()

# Объединяем полигоны районов в одну границу
boundary = gpd.GeoDataFrame(geometry=[districts_gdf.unary_union], crs=districts_gdf.crs)

# ------------------------------------------------------------------------------
# Извлечение объектов: вода, дороги, железные дороги
# ------------------------------------------------------------------------------
# Определяем теги для извлечения объектов
tags = {
    'roads': {
      "highway": [
          "construction", "crossing", "living_street", "motorway", "motorway_link",
          "motorway_junction", "pedestrian", "primary", "primary_link", "raceway",
          "residential", "road", "secondary", "secondary_link", "services",
          "tertiary", "tertiary_link", "track", "trunk", "trunk_link",
          "turning_circle", "turning_loop", "unclassified"
      ],
      "service": ["living_street", "emergency_access"]
    },
    'railways': {
      "railway": "rail"
    },
    'water': {
      'riverbank': True,
      'reservoir': True,
      'basin': True,
      'dock': True,
      'canal': True,
      'pond': True,
      'natural': ['water', 'bay'],
      'waterway': ['river', 'canal', 'ditch'],
      'landuse': 'basin',
      'water': 'lake'
    }
}

# Извлекаем объекты внутри объединённой границы (используем districts_gdf2 для унификации)
water = ox.features_from_polygon(districts_gdf2.unary_union, tags['water'])
roads = ox.features_from_polygon(districts_gdf2.unary_union, tags['roads'])
railways = ox.features_from_polygon(districts_gdf2.unary_union, tags['railways'])

# ------------------------------------------------------------------------------
# Приведение слоёв к локальной системе координат (UTM)
# ------------------------------------------------------------------------------
# Определяем локальную систему координат для точных измерений
local_crs = districts_gdf2.estimate_utm_crs()

# Переводим все слои в локальную систему координат
boundary = boundary.to_crs(local_crs)
water = water.reset_index()[['geometry']].to_crs(local_crs)
roads = roads.reset_index()[['geometry']].to_crs(local_crs)
railways = railways.reset_index()[['geometry']].to_crs(local_crs)

# Фильтруем дороги: оставляем только линейные геометрии (LineString, MultiLineString)
roads = roads[roads.geom_type.isin(['LineString', 'MultiLineString'])].copy()

# ------------------------------------------------------------------------------
# Извлечение зданий и генерация кварталов
# ------------------------------------------------------------------------------
# Извлекаем здания внутри объединённой границы, затем переводим их в точки (центроид)
buildings = ox.features_from_polygon(boundary.to_crs(4326).unary_union, {'building': True})
buildings = buildings.to_crs(local_crs).reset_index()[['geometry']]
buildings.geometry = buildings.representative_point()

# Генерация кварталов (blocks) с использованием BlocksGenerator
bg = BlocksGenerator(boundary, roads, railways, water)
blocks = bg.run()

# Разбиение кварталов с учетом расположения зданий (BlocksSplitter)
bs = BlocksSplitter(blocks, buildings)
splitted_blocks = bs.run()

# ------------------------------------------------------------------------------
# Построение и визуализация графа автомобильных дорог
# ------------------------------------------------------------------------------
# Получаем границу для построения графа по заданному OSM ID
bounds = get_boundary(osm_id=337422)
# Извлекаем граф дорожной сети с дополнительными данными
G_drive = get_drive_graph(polygon=bounds, additional_edgedata=['highway', 'maxspeed', 'reg', 'ref', 'name'])
# Исправляем граф (это необходимо для корректной работы алгоритмов)
AccessibilityProcessor._fix_graph(G_drive)

# Визуализируем граф
fig, ax = ox.plot_graph(
    G_drive,
    node_color='black',   # цвет узлов
    edge_color='grey',      # цвет рёбер
    bgcolor='white',        # цвет фона
    node_size=2,            # размер узлов
    edge_linewidth=2,       # толщина рёбер
    figsize=(25,25)         # размер рисунка
)

# ------------------------------------------------------------------------------
# Дополнительная обработка и проверка данных
# ------------------------------------------------------------------------------
# Убедимся, что слой границ, дорог, водных объектов и железных дорог не пустой
assert not roads.empty, "Ошибка: слой дорог пуст!"
assert not boundary.empty, "Ошибка: слой границ пуст!"
assert not water.empty, "Ошибка: слой водных объектов пуст!"
assert not railways.empty, "Ошибка: слой железных дорог пуст!"

# Если у слоя дорог отсутствует колонка 'highway', создаём заглушку
if 'highway' not in roads.columns:
    roads['highway'] = 'unknown'
# Оставляем только необходимые столбцы в слое дорог
roads = roads[['geometry', 'highway']].copy()

# ------------------------------------------------------------------------------
# Альтернативная визуализация: интерактивное исследование границы
# ------------------------------------------------------------------------------
boundary.explore()


In [ ]:

# ------------------------------------------------------------------------------
# Загрузка зданий в пределах границ и преобразование координат
# ------------------------------------------------------------------------------
# Скачиваем здания, предварительно переводя границу обратно в EPSG:4326 (требуется для OSM)
buildings_raw = ox.features_from_polygon(
    boundary.to_crs(epsg=4326).unary_union,
    tags={"building": True}
)

# Преобразуем здания обратно в локальную проекцию для анализа
buildings = buildings_raw.to_crs(local_crs).reset_index(drop=True)

# Для дальнейшей работы (например, нарезки кварталов) используем representative_point() —
# это точка, гарантированно находящаяся внутри полигона (в отличие от centroid)
buildings['geometry'] = buildings.geometry.representative_point()

# ------------------------------------------------------------------------------
# Нарезка кварталов по зданиям
# ------------------------------------------------------------------------------
# Используем BlocksSplitter для разбиения блоков с учётом зданий
bs = BlocksSplitter(blocks, buildings)
splitted_blocks = bs.run()

# ------------------------------------------------------------------------------
# Визуальное сравнение блоков до и после нарезки
# ------------------------------------------------------------------------------
fig, ax = plt.subplots(1, 2, figsize=(16, 8))

# Отображаем оригинальные кварталы
blocks.plot(ax=ax[0], linewidth=0.1)
ax[0].set_title("Изначальные кварталы")
ax[0].set_axis_off()

# Отображаем кварталы после нарезки
splitted_blocks.plot(ax=ax[1], linewidth=0.1)
ax[1].set_title("Кварталы после нарезки по зданиям")
ax[1].set_axis_off()

# ------------------------------------------------------------------------------
# Проверка и корректировка land_use
# ------------------------------------------------------------------------------
# Проверяем, является ли результат GeoDataFrame (на всякий случай)
if not isinstance(splitted_blocks, gpd.GeoDataFrame):
    splitted_blocks = gpd.GeoDataFrame(splitted_blocks, geometry='geometry', crs="EPSG:3857")

# Проверяем наличие колонки 'land_use'. Если её нет — пробуем заменить на аналогичную
expected_column = 'land_use'
alternative_columns = ['use_type', 'category', 'zone_type']
if expected_column not in splitted_blocks.columns:
    for alt_col in alternative_columns:
        if alt_col in splitted_blocks.columns:
            splitted_blocks = splitted_blocks.rename(columns={alt_col: expected_column})
            break
    else:
        # Если ничего не найдено — задаём дефолтное значение 'residential'
        allowed_values = ['residential', 'business', 'recreation', 'special', 'industrial', 'agriculture', 'transport']
        splitted_blocks[expected_column] = 'residential'

# Убедимся, что все значения land_use входят в допустимый список
allowed_values = {'residential', 'business', 'recreation', 'special', 'industrial', 'agriculture', 'transport'}
invalid_values = set(splitted_blocks[expected_column].unique()) - allowed_values
if invalid_values:
    # Заменяем недопустимые значения на 'residential'
    splitted_blocks.loc[~splitted_blocks[expected_column].isin(allowed_values), expected_column] = 'residential'

# ------------------------------------------------------------------------------
# Расчёт матрицы доступности между кварталами
# ------------------------------------------------------------------------------
# Создаём процессор доступности
ap = AccessibilityProcessor(splitted_blocks)

# Рассчитываем матрицу доступности на основе графа дорожной сети
acc_mx = ap.get_accessibility_matrix(G_drive)

# ------------------------------------------------------------------------------
# Создание модели города
# ------------------------------------------------------------------------------
# Создаём объект City, содержащий кварталы и матрицу доступности между ними
city_model = City(
    blocks=splitted_blocks,
    acc_mx=acc_mx
)

# ------------------------------------------------------------------------------
# Альтернатива: пересчёт графа по полигону, если граница изменилась
# ------------------------------------------------------------------------------
# Получаем границу в проекции EPSG:4326 для запроса в OpenStreetMap
polygon_4326 = boundary.to_crs(epsg=4326).unary_union

# Получаем граф автомобильных дорог с дополнительной информацией
G_drive = get_drive_graph(polygon=polygon_4326, additional_edgedata=['highway', 'maxspeed', 'name'])

# Исправляем граф для совместимости (важный шаг!)
AccessibilityProcessor._fix_graph(G_drive)

# ------------------------------------------------------------------------------
# Визуализация графа дорожной сети
# ------------------------------------------------------------------------------
fig, ax = ox.plot_graph(
    G_drive,
    node_size=2,
    edge_linewidth=1,
    figsize=(12, 12)
)
plt.show()

# ------------------------------------------------------------------------------
# Конвертация графа в GeoDataFrame для дальнейшего анализа
# ------------------------------------------------------------------------------
# Преобразуем граф в GeoDataFrame: узлы и рёбра
n, e = ox.graph_to_gdfs(G_drive)

In [ ]:
# ------------------------------------------------------------------------------
# Инициализация процессора доступности по кварталам
# ------------------------------------------------------------------------------
# Используем `splitted_blocks` напрямую, не через city_model
ap = AccessibilityProcessor(splitted_blocks)

# Расчёт матрицы доступности между кварталами на основе графа дорожной сети
acc_mx = ap.get_accessibility_matrix(G_drive)

# ------------------------------------------------------------------------------
# Добавление матрицы в модель города
# ------------------------------------------------------------------------------
# Обновляем существующий объект city_model, добавляя в него рассчитанную матрицу
city_model.acc_mx = acc_mx

# ------------------------------------------------------------------------------
# Создание объекта для анализа доступности
# ------------------------------------------------------------------------------
accessibility = Accessibility(city_model=city_model)

# ------------------------------------------------------------------------------
# Определение "центрального" квартала — рядом с Казанским собором
# ------------------------------------------------------------------------------
# Координаты Казанского собора (долгота, широта) в системе WGS 84 (EPSG:4326)
kazansky_sobor_coords = Point(30.324444, 59.934167)

# Получаем текущую систему координат кварталов
blocks_gdf = city_model.get_blocks_gdf()
target_crs = blocks_gdf.crs

# Преобразуем координаты собора в систему координат модели города (как пример центрального места возле которого есть памятники и здания)
kazansky_sobor_coords = gpd.GeoSeries([kazansky_sobor_coords], crs="EPSG:4326").to_crs(target_crs).iloc[0]

# Находим квартал, чья геометрия (центроид) ближе всего к точке собора
central_block = min(
    city_model.blocks,
    key=lambda b: b.geometry.centroid.distance(kazansky_sobor_coords)
)

# ------------------------------------------------------------------------------
# Расчёт доступности от выбранного (центрального) квартала
# ------------------------------------------------------------------------------
result_accessibility = accessibility.calculate(central_block)

# ------------------------------------------------------------------------------
# Визуализация доступности от Казанского собора
# ------------------------------------------------------------------------------
# Строим карту: доступность окрашивается градиентом
Accessibility.plot(result_accessibility, linewidth=0.9, figsize=(21, 7))

# Получаем текущую ось для добавления дополнительных объектов
ax = plt.gca()

# Добавляем на карту обводку квартала, где находится Казанский собор
gpd.GeoSeries([central_block.geometry], crs=target_crs).plot(
    ax=ax, edgecolor='blue', facecolor="none", linewidth=2, label="Квартал Центра"
)

# Добавляем маркер в центре этого квартала
ax.scatter(
    central_block.geometry.centroid.x,
    central_block.geometry.centroid.y,
    color='red',
    s=100,
    marker='o',
    label="Центр"
)

# ------------------------------------------------------------------------------
# Завершающее оформление визуализации
# ------------------------------------------------------------------------------
ax.set_title("Доступность от Казанского собора", fontsize=14)
ax.legend()
plt.show()

In [ ]:
# ------------------------------------------------------------------------------
# Проверка наличия столбца 'Score' в GeoDataFrame кандидатов
# ------------------------------------------------------------------------------
if "Score" not in candidates.columns:
    raise ValueError("Ошибка: В GeoDataFrame candidates нет столбца 'Score'.")

# ------------------------------------------------------------------------------
# Подготовка данных для визуализации
# ------------------------------------------------------------------------------
# Создаем копию, чтобы не изменять исходный GeoDataFrame
candidates_vis = candidates.copy()

# Заменяем бесконечности на NaN и заполняем их медианным значением
candidates_vis["Score"] = candidates_vis["Score"].replace([np.inf, -np.inf], np.nan)
candidates_vis["Score"] = candidates_vis["Score"].fillna(candidates_vis["Score"].median())

# Проверяем корректность минимального и максимального значений
vmin, vmax = candidates_vis["Score"].min(), candidates_vis["Score"].max()
if vmin >= vmax or np.isnan(vmin) or np.isnan(vmax):
    raise ValueError(f"Ошибка: некорректные границы [{vmin}, {vmax}].")

# ------------------------------------------------------------------------------
# Обрезка крайних значений (1–99 процентили) для более стабильной цветовой шкалы
# ------------------------------------------------------------------------------
q_low, q_high = candidates_vis["Score"].quantile([0.01, 0.99])
candidates_vis["Score"] = np.clip(candidates_vis["Score"], q_low, q_high)

# ------------------------------------------------------------------------------
# Приведение данных к географической проекции для корректного отображения
# ------------------------------------------------------------------------------
candidates_vis_4326 = candidates_vis.to_crs(epsg=4326)
selected_objects_4326 = selected_objects.to_crs(epsg=4326)

# ------------------------------------------------------------------------------
# Создание статичной карты с визуализацией Score
# ------------------------------------------------------------------------------
fig, ax = plt.subplots(figsize=(12, 10))

# Отображаем всех кандидатов, закрашенных по значению 'Score'
candidates_vis_4326.plot(
    ax=ax,
    column='Score',              # Окрашиваем по значению Score
    cmap='Blues',                # Цветовая палитра
    legend=True,                 # Добавляем легенду
    markersize=6,                # Размер точек
    label='Кандидаты',
    legend_kwds={
        'label': "Score",
        'shrink': 0.6            # Уменьшаем размер легенды
    }
)

# Отображаем выбранные объекты поверх — красным цветом
selected_objects_4326.plot(
    ax=ax,
    color='red',
    markersize=1,
    label='Выбранные объекты'
)

# ------------------------------------------------------------------------------
# Добавление фоновой карты от CartoDB
# ------------------------------------------------------------------------------
ctx.add_basemap(
    ax,
    crs=candidates_vis_4326.crs,
    source=ctx.providers.CartoDB.Positron
)

# ------------------------------------------------------------------------------
# Финальное оформление визуализации
# ------------------------------------------------------------------------------
ax.set_title("Оценка связности (Score) и расположение объектов", fontsize=14)
ax.set_axis_off()   # Убираем оси координат
plt.legend()        # Добавляем легенду
plt.show()